# Validation

In [1]:

import os
import sys
import numpy as np
import pandas as pd
import random

from argparse import ArgumentParser
import datetime

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.backends.cudnn as cudnn

from pytvision.transforms import transforms as mtrans
from pytvision import visualization as view

sys.path.append('../')
from torchlib.datasets  import factory
from torchlib.datasets  import gdata
from torchlib.neuralnet import NeuralNet
from torchlib.datasets.imaterialist import IMaterialistImageDataset

from misc import get_transforms_aug, get_transforms_det, get_transforms_hflip, get_transforms_gray, get_transforms_aug2
from sklearn import metrics


sys.version_info(major=3, minor=6, micro=5, releaselevel='final', serial=0)


In [2]:
home = os.getenv("HOME")
pathname= os.path.join(home, '.datasets')
name_dataset='imaterialist'
project='../out/netruns'
no_cuda=False
seed=1
gpu=1
name='exp_net_baseline_resnet152_imaterialist_001'
model= 'chk000050.pth.tar' #'model_best.pth.tar'
path_model=os.path.join(project,name,'models',model)
batch_size=120
workers=80
num_input_channels=3
pathnameout = os.path.join(project, 'validation')

print(pathname)
print(project)
print(name_dataset)
print(path_model)


/home/fernandez/.datasets
../out/netruns
imaterialist
../out/netruns/exp_net_baseline_resnet152_imaterialist_001/models/chk000050.pth.tar


In [3]:

# neuralnet
network = NeuralNet(
    patchproject=project,
    nameproject=name,
    no_cuda=no_cuda,
    seed=seed,
    gpu=gpu
    )

cudnn.benchmark = True

# load model
if network.load( path_model ) is not True:
    assert(False)

tta_preprocess = [ 
    get_transforms_det(network.size_input), 
    get_transforms_hflip(network.size_input), 
    #get_transforms_gray(network.size_input),
    #get_transforms_aug(network.size_input),
    #get_transforms_aug2(network.size_input)
    ]

dataloaders = []
for transform in tta_preprocess:    
    # test dataset
    data = gdata.Dataset(
        data=factory.FactoryDataset.factory(
            pathname=pathname, 
            name=name_dataset, 
            subset=factory.validation, 
            download=True ),
        num_channels=num_input_channels,
        transform=transform
        )
    dataloader = DataLoader(data, batch_size=batch_size, shuffle=False, num_workers=workers )
    dataloaders.append(dataloader)

print(len(dataloaders))
    

=> loading checkpoint '../out/netruns/exp_net_baseline_resnet152_imaterialist_001/models/chk000050.pth.tar'
=> loaded checkpoint for resnet152 arch!
2


In [4]:
random.seed( seed )
files = [ f for f in sorted(os.listdir(pathnameout)) if f.split('.')[-1] == 'csv' ]; 

for i,data in enumerate(dataloaders):
    Yhat, Y = network.test( data )
    df = pd.DataFrame( np.concatenate((Y, Yhat), axis=1) )
    df.to_csv( os.path.join(pathnameout , 'val_{}_dp{}.csv'.format(name, i + len(files) )  ), index=False, encoding='utf-8')        

print('dir: {}'.format(pathnameout))
print('DONE!!!')


100%|██████████| 53/53 [00:32<00:00,  1.63it/s]


dir: ../out/netruns/validation
DONE!!!


 # Multiclasification systems
 
       +-----+  dp_1
     --| cnn |---------+
       +-----+         |
                       | 
       +-----+  dp_2   |     +--------+
     --| cnn |---------+ ----| Fusion |-----------+
       +-----+         |     +--------+
                       .        |
                       .        SOFT, HARD, TR
                       .        Soft: sRp, sRs, sRmx, sRmi, sRmd, sRmv
       +-----+  dp_L   |        Hard: hWmv, hRec, hNb  
     --| cnn |---------+        Train: tFi, tLop, tMb, tMsvm
       +-----+


In [5]:

## On Combining Classifiers
# - Combining soft ruler
# - https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    
def product_ruler( dp, P=None ):
    """
    Ecuation. Josef Kittler [7]
    https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    Soft Product Rule    
    P^{(-(R-1))}( w_j )\prod_iP(w_j|x_i)  = \max_k P^{(-(R-1))}( w_k ) \prod_i
    P(w_k|x_i) (1)
    which under the assumption of equal priors, simplifies to the following:
    \prod_iP(w_j|x_i)  = \max_k \prod_i P(w_k|x_i) (2)    
    Args:
        @dp: []_nxcxl
        @P: class prior []_c
    """    
    p = dp.prod(2)
    if P is not None:
        l = dp.shape[2]
        p = P**-(l-1)*p
    return p

def sum_ruler( dp, P=None ):
    """
    Ecuation. Josef Kittler [11]
    https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    Soft Sum Ruler
    $$(1-R)P( w_j ) + \sum_iP(w_j|x_i)  = \max_k [ (1-R)P(w_k) + \sum_iP(w_k|x_i)]$$
    which under the assumption of equal priors simplifies to the following:
    $$\sum_iP(w_j|x_i)  = \max_k \sum_iP(w_k|x_i)$$    
    Args:
        @dp: []_nxcxl
    """
    p = dp.sum(2)
    if P is not None:
        l = dp.shape[2]
        p = (1-l)*P + p
    return p

def max_ruler( dp, P=None ):
    """
    Ecuation. Josef Kittler [14][15]
    https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    Soft Max Ruler
    Args:
        @dp: []_nxcxl
    """
    p = dp.max(2)
    if P is not None:
        l = dp.shape[2]
        p = (1-l)*P + l*p
    return p

def min_ruler( dp, P=None ):
    """
    Ecuation. Josef Kittler [16]
    https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    Soft Min Ruler
    Args:
        @dp: []_nxcxl
    """
    p = dp.min(2)
    if P is not None:
        l = dp.shape[2]
        p = P**-(l-1)*p
        p = (1-l)*P + l*p
    return p


def majority_ruler( dp  ):
    """
    Ecuation. Josef Kittler [20]
    https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    Soft Majority Vote Rule
    Args:
        @dp: []_nxcxl
    """
    n,c,l = dp.shape
    p = np.argmax(dp,axis=1)
    
    dki = np.zeros((n,c))
    for i in range(n):
        tup = p[i,:]
        for j in range(c):
            dki[i,j] = np.sum( tup == j )
        
    p=dki
    return p


def mean_ruler( dp ):
    """
    Ecuation. Josef Kittler [18]
    https://dspace.cvut.cz/bitstream/handle/10467/9443/1998-On-combining-classifiers.pdf?sequence=1
    Soft Median Ruler
    Args:
        @dp: []_nxcxl
    """
    p = dp.mean(2)
    return p
    

    
# # test
# #[n,c,l]
# #dp = np.random.rand(10,4,3 )
# #P = np.array([0.1,0.1,0.1,0.7])
# dp = np.array( [[[0.2,0,0,0],[0.8,1.0,1.0,1.0]],[[0.3,0.6,0.9,0.5],[0.7,0.4,0.1,0.5]]] )
# P = np.array([0.7,0.3])
# print(dp.shape)
# #print(dp[:,:,0])

# func = [product_ruler, sum_ruler, max_ruler, min_ruler, majority_ruler, mean_ruler]
# for f in func:
#     p = f(dp)
#     print( p.argmax(1) )
    
    

In [18]:
files = np.array([ f for f in sorted(os.listdir(pathnameout)) if f.split('.')[-1] == 'csv' ]); 
#idx = np.array([0,1,2,3,6,7], dtype=int) #2,3,7
# files = files[:-2]
print(files)

l = len(files)
dp =[]; ys=[]
for f in files:  
    mdata = pd.read_csv( os.path.join(pathnameout , f )  )
    dpdata = mdata.as_matrix()
    ys.append(dpdata[:,0])    
    dp.append( dpdata[:,1:] )
    
dp = np.array(dp).transpose((1,2,0))
ys = np.array(ys)

assert( not (ys[0,:]-ys[:1,:]).sum() )
print(dp.shape)
    

['val_exp_net_baseline_nasnetalarge_imaterialist_002_dp0.csv'
 'val_exp_net_baseline_nasnetalarge_imaterialist_002_dp1.csv'
 'val_exp_net_baseline_nasnetalarge_imaterialist_002_dp2.csv'
 'val_exp_net_baseline_nasnetalarge_imaterialist_002_dp3.csv'
 'val_exp_net_baseline_resnet152_imaterialist_001_dp6.csv'
 'val_exp_net_baseline_resnet152_imaterialist_001_dp7.csv']
(6309, 128, 6)


In [19]:
y = ys[0,:]

# individual result
print('\nIndividual: ')
p = np.argmax(dp,axis=1)
for i in range(p.shape[1]):
    pred = p[:,i]
    acc = (pred==y).astype(float).sum()/len(y)
    print(i, '{}\tmodel_{}:\t{}'.format(files[i], i, acc) )

# multiclasification result
print('\nMulticlasification: ')
func = [product_ruler, sum_ruler, max_ruler, min_ruler, majority_ruler, mean_ruler]
for f in func:
    p = f(dp)
    pred = np.argmax(p, axis=1)
    acc = (pred==y).astype(float).sum()/len(y)
    print('{}:\t{}'.format(f.__name__, acc))


# p = max_ruler(dp)
# pred = np.argmax(p, axis=1)
# print(y[:10], y.sum() )
# print(pred[:10], pred.sum() )
# acc = (pred==y).sum()/len(y)
# print(acc)



Individual: 
0 val_exp_net_baseline_nasnetalarge_imaterialist_002_dp0.csv	model_0:	0.8464098906324299
1 val_exp_net_baseline_nasnetalarge_imaterialist_002_dp1.csv	model_1:	0.8505309874782058
2 val_exp_net_baseline_nasnetalarge_imaterialist_002_dp2.csv	model_2:	0.8441908384847044
3 val_exp_net_baseline_nasnetalarge_imaterialist_002_dp3.csv	model_3:	0.8429228086860041
4 val_exp_net_baseline_resnet152_imaterialist_001_dp6.csv	model_4:	0.8167696940878111
5 val_exp_net_baseline_resnet152_imaterialist_001_dp7.csv	model_5:	0.8202567760342369

Multiclasification: 
product_ruler:	0.8678078934854969
sum_ruler:	0.8663813599619591
max_ruler:	0.8579806625455698
min_ruler:	0.8614677444919956
majority_ruler:	0.8625772705658583
mean_ruler:	0.8663813599619591
